# Data Preparation
find number of suitable book

## load data

In [8]:
import json
import pandas as pd

# Load the goodreads_book_works.json file (JSONL format - one JSON object per line)
data = []
with open('./data/goodreads_book_works.json', 'r', encoding='utf-8') as f:
    for line in f:
        if line.strip():
            data.append(json.loads(line))

# Convert to DataFrame for easier analysis
df = pd.DataFrame(data)

# Display the shape and columns
print(f"Total records: {len(df)}")
print(f"Columns: {df.columns.tolist()}")
print(f"\nFirst few rows:")
print(df.head())

Total records: 1521962
Columns: ['books_count', 'reviews_count', 'original_publication_month', 'default_description_language_code', 'text_reviews_count', 'best_book_id', 'original_publication_year', 'original_title', 'rating_dist', 'default_chaptering_book_id', 'original_publication_day', 'original_language_id', 'ratings_count', 'media_type', 'ratings_sum', 'work_id']

First few rows:
  books_count reviews_count original_publication_month  \
0           1             6                          8   
1          22         10162                              
2           2           268                              
3          38         89252                          7   
4           2            49                              

  default_description_language_code text_reviews_count best_book_id  \
0                                                    1      5333265   
1                                                  741        25717   
2                                                 

## data inspect

In [9]:
# First, let's check the structure of the reviews file
print("Checking structure of goodreads_reviews_dedup.json...")
with open('./data/goodreads_reviews_dedup.json', 'r', encoding='utf-8') as f:
    for i in range(3):
        line = f.readline()
        if line.strip():
            review = json.loads(line)
            print(f"\nSample review {i+1}:")
            print(f"Keys: {review.keys()}")
            print(f"Content: {review}")
            break


Checking structure of goodreads_reviews_dedup.json...

Sample review 1:
Keys: dict_keys(['user_id', 'book_id', 'review_id', 'rating', 'review_text', 'date_added', 'date_updated', 'read_at', 'started_at', 'n_votes', 'n_comments'])
Content: {'user_id': '8842281e1d1347389f2ab93d60773d4d', 'book_id': '24375664', 'review_id': '5cd416f3efc3f944fce4ce2db2290d5e', 'rating': 5, 'review_text': "Mind blowingly cool. Best science fiction I've read in some time. I just loved all the descriptions of the society of the future - how they lived in trees, the notion of owning property or even getting married was gone. How every surface was a screen. \n The undulations of how society responds to the Trisolaran threat seem surprising to me. Maybe its more the Chinese perspective, but I wouldn't have thought the ETO would exist in book 1, and I wouldn't have thought people would get so over-confident in our primitive fleet's chances given you have to think that with superior science they would have weapons

## review count

In [10]:
# Count actual reviews per book from the reviews file
from collections import Counter

print("Counting actual reviews per book from goodreads_reviews_dedup.json...")

review_counts = Counter()
lines_processed = 0

with open('./data/goodreads_reviews_dedup.json', 'r', encoding='utf-8') as f:
    for line in f:
        lines_processed += 1
        if lines_processed % 100000 == 0:
            print(f"Processed {lines_processed} lines...")
        
        if line.strip():
            try:
                review = json.loads(line)
                book_id = int(review.get('book_id', -1))
                review_counts[book_id] += 1
            except:
                pass

print(f"Total lines processed: {lines_processed}\n")

# Find books with actual review count > 1250
target = 1250
books_above_target = {book_id: count for book_id, count in review_counts.items() 
                      if count > target}

print(f"Books with actual review count > {target}: {len(books_above_target)}\n")

print("=" * 100)
print(f"Top 20 books by actual review count (> {target}):\n")

# Sort and display top books
sorted_books = sorted(books_above_target.items(), key=lambda x: x[1], reverse=True)
for idx, (book_id, count) in enumerate(sorted_books[:20], 1):
    print(f"{idx:2d}. Book ID {book_id}: {count} reviews")

print("\n" + "=" * 100)
print(f"\nTotal books with > {target} reviews: {len(books_above_target)}")
print(f"Review count range: {min(books_above_target.values())} to {max(books_above_target.values())}")


Counting actual reviews per book from goodreads_reviews_dedup.json...
Processed 100000 lines...
Processed 200000 lines...
Processed 300000 lines...
Processed 400000 lines...
Processed 500000 lines...
Processed 600000 lines...
Processed 700000 lines...
Processed 800000 lines...
Processed 900000 lines...
Processed 1000000 lines...
Processed 1100000 lines...
Processed 1200000 lines...
Processed 1300000 lines...
Processed 1400000 lines...
Processed 1500000 lines...
Processed 1600000 lines...
Processed 1700000 lines...
Processed 1800000 lines...
Processed 1900000 lines...
Processed 2000000 lines...
Processed 2100000 lines...
Processed 2200000 lines...
Processed 2300000 lines...
Processed 2400000 lines...
Processed 2500000 lines...
Processed 2600000 lines...
Processed 2700000 lines...
Processed 2800000 lines...
Processed 2900000 lines...
Processed 3000000 lines...
Processed 3100000 lines...
Processed 3200000 lines...
Processed 3300000 lines...
Processed 3400000 lines...
Processed 3500000 lin

## genre

In [ ]:
# Load genre data and find 8 books with different genres from books_above_target
import json
from collections import defaultdict

print("Loading genre data from goodreads_book_genres_initial.json...")

# Build a mapping of book_id -> genres
book_genres_map = {}

with open('./data/goodreads_book_genres_initial.json', 'r', encoding='utf-8') as f:
    for line in f:
        if line.strip():
            try:
                entry = json.loads(line)
                book_id = int(entry.get('book_id', -1))
                genres = entry.get('genres', {})
                book_genres_map[book_id] = genres
            except:
                pass

print(f"Loaded genres for {len(book_genres_map)} books\n")

# Find books from books_above_target that have genre data
books_with_genres = {}
for book_id in books_above_target.keys():
    if book_id in book_genres_map:
        books_with_genres[book_id] = book_genres_map[book_id]

print(f"Books from books_above_target with genre data: {len(books_with_genres)}\n")

# Select 8 books with different genres
# Strategy: group by primary genre and pick one from each group
genre_to_books = defaultdict(list)

for book_id, genres in books_with_genres.items():
    if isinstance(genres, dict) and genres:  # genres is a dict
        # Get the first (primary) genre from the dict keys
        primary_genre = list(genres.keys())[0]
        genre_to_books[primary_genre].append(book_id)
    elif isinstance(genres, list) and genres:  # In case genres is a list
        primary_genre = genres[0]
        genre_to_books[primary_genre].append(book_id)

print("Genre distribution (top 20):")
sorted_genres_list = sorted(genre_to_books.keys(), key=lambda g: len(genre_to_books[g]), reverse=True)
for genre in sorted_genres_list[:20]:
    print(f"  {genre}: {len(genre_to_books[genre])} books")

# Select 8 books with different genres (one per genre, prioritizing genres with fewer books)
books_genre_1250_above_ids = []
selected_genres = []

# Sort genres by number of books (pick from genres with fewer books first for diversity)
sorted_genres_by_count = sorted(genre_to_books.keys(), key=lambda g: len(genre_to_books[g]))

for genre in sorted_genres_by_count:
    if len(books_genre_1250_above_ids) < len(sorted_genres_list):
        book_id = genre_to_books[genre][0]
        books_genre_1250_above_ids.append(book_id)
        selected_genres.append(genre)

print(f"\n{'=' * 100}")
print(f"Selected 10 books with different genres:\n")

for idx, (book_id, genre) in enumerate(zip(books_genre_1250_above_ids, selected_genres), 1):
    review_count = books_above_target[book_id]
    print(f"{idx}. Book ID {book_id}: {review_count} reviews | Primary Genre: {genre}")

print(f"\n{'=' * 100}")
print(f"\nbooks_genre_1250_above_ids = {books_genre_1250_above_ids}")


Loading genre data from goodreads_book_genres_initial.json...
Loaded genres for 2360655 books

Books from books_above_target with genre data: 726

Genre distribution (top 20):
  fiction: 260 books
  young-adult: 159 books
  fantasy, paranormal: 157 books
  romance: 59 books
  non-fiction: 50 books
  mystery, thriller, crime: 26 books
  comics, graphic: 7 books
  children: 4 books
  history, historical fiction, biography: 3 books
  poetry: 1 books

Selected 8 books with different genres:

1. Book ID 23513349: 2814 reviews | Primary Genre: poetry
2. Book ID 9938498: 1518 reviews | Primary Genre: history, historical fiction, biography
3. Book ID 3636: 6156 reviews | Primary Genre: children
4. Book ID 18659623: 1493 reviews | Primary Genre: comics, graphic
5. Book ID 6218281: 1952 reviews | Primary Genre: mystery, thriller, crime
6. Book ID 27161156: 1911 reviews | Primary Genre: non-fiction
7. Book ID 15507958: 9557 reviews | Primary Genre: romance
8. Book ID 136251: 6486 reviews | Primar

## extract review
from chosen book

In [12]:
import json
from collections import Counter

# Extract reviews for the 8 books with different genres (actual review count > 1250)
print(f"Extracting reviews for {len(books_genre_1250_above_ids)} books with different genres (actual review count > 1250)...\n")

reviews_1250_above = []
review_count = 0
lines_processed = 0

input_file = './data/goodreads_reviews_dedup.json'
output_file = './data/books_genre_1250_above_reviews.json'

with open(input_file, 'r', encoding='utf-8') as f:
    for line in f:
        lines_processed += 1

        # Progress print
        if lines_processed % 100000 == 0:
            print(f"Processed {lines_processed:,} lines, found {review_count:,} matches...")

        line = line.strip()
        if not line:
            continue

        try:
            review = json.loads(line)
        except json.JSONDecodeError:
            continue

        review_book_id = int(review.get('book_id', -1))

        if review_book_id in books_genre_1250_above_ids:
            # Filter out reviews containing "..." (truncated reviews)
            review_text = review.get('review_text', '')
            if review_text and '...' not in review_text:
                reviews_1250_above.append(review)
                review_count += 1

print(f"\nTotal reviews found: {review_count:,}\n")

# ------------------------------------------
# SAVE AS VALID JSON ARRAY
# ------------------------------------------

with open(output_file, 'w', encoding='utf-8') as f:
    f.write('[\n')
    for idx, review in enumerate(reviews_1250_above):
        f.write(json.dumps(review, ensure_ascii=False))
        if idx < len(reviews_1250_above) - 1:
            f.write(',\n')
    f.write('\n]')

print(f"Successfully saved {review_count:,} reviews to '{output_file}'\n")

# ------------------------------------------
# SUMMARY BY BOOK
# ------------------------------------------

if review_count > 0:
    print("=" * 120)
    print("Summary of reviews by book:\n")

    book_review_counts = Counter()

    for review in reviews_1250_above:
        book_id = int(review.get('book_id', -1))
        book_review_counts[book_id] += 1

    # Display sorted by book_id
    for book_id in sorted(book_review_counts.keys()):
        count = book_review_counts[book_id]
        print(f"  Book ID {book_id}: {count} reviews")


Extracting reviews for 10 books with different genres (actual review count > 1250)...

Processed 100,000 lines, found 250 matches...
Processed 200,000 lines, found 500 matches...
Processed 300,000 lines, found 755 matches...
Processed 400,000 lines, found 1,007 matches...
Processed 500,000 lines, found 1,263 matches...
Processed 600,000 lines, found 1,535 matches...
Processed 700,000 lines, found 1,791 matches...
Processed 800,000 lines, found 2,044 matches...
Processed 900,000 lines, found 2,296 matches...
Processed 1,000,000 lines, found 2,567 matches...
Processed 1,100,000 lines, found 2,821 matches...
Processed 1,200,000 lines, found 3,097 matches...
Processed 1,300,000 lines, found 3,342 matches...
Processed 1,400,000 lines, found 3,633 matches...
Processed 1,500,000 lines, found 3,905 matches...
Processed 1,600,000 lines, found 4,165 matches...
Processed 1,700,000 lines, found 4,422 matches...
Processed 1,800,000 lines, found 4,662 matches...
Processed 1,900,000 lines, found 4,93

## JSON to CSV Converter

In [13]:
# Convert JSON to CSV
print("Converting books_1250_above_reviews.json to CSV format...\n")

# Create DataFrame from reviews
df_reviews = pd.DataFrame(reviews_1250_above)

# Select key columns for CSV
columns_to_keep = ['book_id', 'user_id', 'review_id', 'rating', 'review_text', 'date_added', 'date_updated', 'n_votes', 'n_comments']

# Keep only available columns
available_cols = [col for col in columns_to_keep if col in df_reviews.columns]
df_reviews_csv = df_reviews[available_cols]

# Save to CSV
csv_output_file = './data/books_1250_above_reviews.csv'
df_reviews_csv.to_csv(csv_output_file, index=False, encoding='utf-8')

print(f"Successfully converted to CSV!")
print(f"File: {csv_output_file}")
print(f"Total rows: {len(df_reviews_csv):,}")
print(f"Columns: {', '.join(available_cols)}\n")

# Display sample
print("=" * 120)
print("Sample of CSV data (first 5 rows):\n")
print(df_reviews_csv.head().to_string(index=False))

# File size comparison
import os
json_size = os.path.getsize('./data/books_1250_above_reviews.json') / (1024*1024)
csv_size = os.path.getsize(csv_output_file) / (1024*1024)

print(f"\n{'=' * 120}")
print(f"File Size Comparison:")
print(f"  JSON: {json_size:.2f} MB")
print(f"  CSV:  {csv_size:.2f} MB")
print(f"  Saved: {(json_size - csv_size):.2f} MB ({((json_size - csv_size) / json_size * 100):.1f}%)")


Converting books_1250_above_reviews.json to CSV format...

Successfully converted to CSV!
File: ./data/books_1250_above_reviews.csv
Total rows: 46,358
Columns: book_id, user_id, review_id, rating, review_text, date_added, date_updated, n_votes, n_comments

Sample of CSV data (first 5 rows):

 book_id                          user_id                        review_id  rating                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 